In [2]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view #for easy minima without using for loops
from math import *

Wisniowski and Rickman (2013): Fast MOID calculation

because Gronchi's code is too complicated


In [ ]:
# Step 1: ~(cover yourself in oil)~ rotate matrix
# Break out the 403 notes: Compute dcm_a, dcm_b, convert from a to b to get dcm_b_a 
# (dcm of b wrt A, where dcm_b is an orientation and a is a )
# Passive rotation

#D_HF = D_HG @ D_GF
#D_BA = D_B0 @ D_0A = D_B0 @ D_A0.T   from A to B
#D_AB = D_A0 @ D_0B
def rotate(inc_a,argpe_a,omega_a,inc_b,argpe_b,omega_b):
    """
    Remember all angles are in radians and negative!!!
    """
    dcm_a = np.array([[ np.cos(omega_a), np.sin(omega_a), 0],
                      [-np.sin(omega_a), np.cos(omega_a), 0],
                      [               0,               0, 1]]) @ np.array([[ 1, 0, 0],
                      [0,  np.cos(inc_a), np.sin(inc_a)],
                      [0, -np.sin(inc_a), np.cos(inc_a)]]) @ np.array([[ np.cos(argpe_a),np.sin(argpe_a), 0],
                      [-np.sin(argpe_a), np.cos(argpe_a), 0],
                      [                         0,                        0, 1]])
    dcm_b = np.array([[ np.cos(omega_b), np.sin(omega_b), 0],
                      [-np.sin(omega_b), np.cos(omega_b), 0],
                      [               0,               0, 1]]) @ np.array([[ np.cos(inc_b), 0, -np.sin(inc_b)],
                      [             0, 1,             0],
                      [ np.sin(inc_a), 0, np.cos(inc_b)]]) @ np.array([[ np.cos(argpe_b),np.sin(argpe_b), 0],
                      [-np.sin(argpe_a), np.cos(argpe_a), 0],
                      [                         0,                        0, 1]])
    dcm_ba = dcm_b@dcm_a.T #dcm from A to B
    # Now we convert this back into Euler using 403 formula
    # in the 403 notes, phi = argpe, theta = inc, psi = omega
    # remember you need to make this negative
    inc_ba = -np.arccos(dcm_ba[2][2])
    argpe_ba = -np.arctan2(dcm_ba[2][0], -dcm_ba[2][1])
    omega_ba = -np.arctan2(dcm_ba[0][2], dcm_ba[1][2]) #I sure hope this works.
    return inc_ba, argpe_ba, omega_ba

lst = [-9, 29, -100, 64, 26, 73, -96, 28, -92, 11, -14.000000000000000000000000000000000000051, -29.9999999992, -36.0001, -35.000005, -86, -36, -54, -67, 8, 10, -20,-6] #test
arr = np.array(lst)
# print(minima_sto(lst))
# print(lst[1:-1])

# to find local minima: D_min_i should be smaller than both preceding and following values
# What this means: Get log of all distances D_squared, run function to detect minima (I'll code this myself)
def minima(D_sq):
    """
    Strategy:
    1) Append 0th element to end of D_sq (for wraparound purposes)
    2) Sliding window view to create giant array of n x 3
    3) Find minima
    4) Knowing center column is either [1: (n+1)] in np.append or D_sq itself in np.pad,
    subtract D_sq from that and find zeros. Indices that are 0 are local minima.
    5) Return indices for easy math
    """
    # D_sq_wrap = np.append(D_sq,[D_sq[0],D_sq[1]])
    # print(np.size(D_sq))
    # print(D_sq_wrap, D_sq.size)
    # print(D_sq)
    # q = sliding_window_view(D_sq_wrap,3) # Produces n-1 size array, slice first element
    # print(q, np.min(q,axis=1))
    # print(np.shape(D_sq[1:]))
    # print(np.min(q,axis=1)-np.take(D_sq,np.arange(1,np.size(D_sq)+1),mode='wrap'))
    # print(np.argwhere(np.min(q,axis=1)-np.take(D_sq,np.arange(1,np.size(D_sq)+1),mode='wrap')).T[0]) 
    # return np.argwhere(np.min(q,axis=1)-np.take(D_sq,np.arange(1,np.size(D_sq)+1),mode='wrap') == 0).T[0] - 1 # Returns indices of D_min

    # D_sq_wrap = np.append(D_sq,[D_sq[0],D_sq[1]])
    # q = sliding_window_view(D_sq_wrap,3) # Produces n-1 size array, slice first element
    # print(np.min(q,axis=1)-np.take(D_sq,np.arange(1,np.size(D_sq)+1),mode='wrap'))
    # return np.argwhere(np.min(q,axis=1)-np.take(D_sq,np.arange(1,np.size(D_sq)+1),mode='wrap') == 0).T[0] - 1 # Returns indices of D_min

    D_sq_wrap = np.pad(D_sq,(1,1),'wrap')
    q = sliding_window_view(D_sq_wrap,3) # Produces n-1 size array, slice first element
    # print(np.min(q,axis=1)-D_sq)
    return np.argwhere((np.min(q,axis=1)-D_sq) == 0 ).T[0] #Returns indices, needed to find L and nu
    # return D_sq[np.argwhere((np.min(q,axis=1)-D_sq) == 0 ).T[0]] #Returns actual values if you so please

minima(arr)
# arr[8]
# %timeit minima(np.random.rand(400))

def moid(a_a,e_a,i_1,argpe_1,raan_1, a_b,e_b,i_2,argpe_2,raan_2):
    i_b, argpe_b, raan_b = rotate(i_1,argpe_1,raan_1,i_2,argpe_2,raan_2)
    # Need to record L and nu for each index
    

array([  -9.    , -100.    ,   26.    ,  -96.    ,  -92.    ,  -36.0001,
        -86.    ,  -67.    ,  -20.    ])

In [ ]:
#inc, argpe, omega
rotate(np.deg2rad(-90),np.deg2rad(0),np.deg2rad(0),np.deg2rad(0),np.deg2rad(0),np.deg2rad(0))

[[ 1.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  6.123234e-17 -1.000000e+00]
 [ 0.000000e+00  1.000000e+00  6.123234e-17]]


In [32]:
print(arr[7:16])

[ 28 -92  11 -14 -86 -36 -54 -67   8]


In [54]:
%timeit minima(arr)

27.1 µs ± 986 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [55]:
%timeit minima(np.random.rand(400))

37.8 µs ± 516 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
